# Семинар 1. Word embeddings
На этом занятии мы познакомимся с несколькими моделями векторных представлений слов: обучим с нуля пару простых моделей, убедимся в том, что в пространстве word2vec векторные операции соответствуют смысловым изменениям, а также попробуем решить с их помощью прикладную задачу sentiment analysis. 

Для первой части воспользуемся набором неразмеченных текстов, часто используемым для моделирования языка — [Wikitext-2](https://blog.einstein.ai/the-wikitext-long-term-dependency-language-modeling-dataset/):  

In [1]:
!wget -q -nc https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-v1.zip
!unzip -o wikitext-2-v1

/bin/sh: wget: command not found
unzip:  cannot find or open wikitext-2-v1, wikitext-2-v1.zip or wikitext-2-v1.ZIP.


Для начала считаем все строки из входного файла (уже разбитого на токены) и приведём их к нижнему регистру.

In [ ]:
with open('wikitext-2/wiki.train.tokens') as f:
    lines = [line.strip().lower() for line in f]

Чтобы подготовить данные для обучения, сначала необходимо построить словарь — отображение из слова в его индекс и наоборот. Также нам впоследствии может пригодиться получение списка самых частых слов, поэтому стоит для каждого слова сохранять число его вхождений в корпус. Пользуясь классом [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter), постройте счётчик вхождений каждого слова в датасет, а также словари word_to_ind и ind_to_word, отображающие слово в целочисленный индекс и наоборот.

In [ ]:
from collections import Counter
vocab = Counter()

for line in lines:
    if line:
        words = line.split()
        vocab.update(words)

word_to_ind = {word : i for i, word in enumerate(vocab)}
ind_to_word = {i : word for word, i in word_to_ind.items()}

Создайте матрицу попарной встречаемости слов cooc_matrix размера `len(vocab) x len(vocab)`, в ячейке i,j которой содержится информация о том, как часто  слова с индексами i и j находились в контексте друг друга. В качестве контекста используйте скользящее окно с центром в каждом слове предложения размера 5.

Bonus tip: строго говоря, более разумно для построения word embeddings использовать матрицу Pointwise Mutual Information (PMI). При желании можно обратиться к [статье](https://papers.nips.cc/paper/2014/file/feab05aa91085b7a8012516bc3533958-Paper.pdf) и реализовать подсчёт этой характеристики — последующие результаты должны стать лучше. 

In [ ]:
import numpy as np
cooc_matrix = np.zeros((len(vocab),len(vocab)),dtype=np.float32)

CONTEXT_WINDOW_SIZE = 5
N_SIDE_NEIGHBOURS = CONTEXT_WINDOW_SIZE / 2

for line in lines:
    words = line.split()
    for i, word in enumerate(words):
        index_for_word_i = word_to_ind[word]
        for shift_index in range(-N_SIDE_NEIGHBOURS, N_SIDE_NEIGHBOURS+1):
            j = i + shift_index
            if (j > 0) and (j < len(words)) and j != i:
                word_j = words[j]
                index_for_word_j = word_to_ind[word_j]
                cooc_matrix[index_for_word_i, index_for_word_j] += 1
                cooc_matrix[index_for_word_j, index_for_word_i] += 1

Теперь мы можем построить простые векторные представления слов: воспользуемся усечённым SVD-разложением и понизим размерность матрицы совстречаемости слов до 300.

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svd = TruncatedSVD(n_components=300, random_state=0)
svd_embeddings = svd.fit_transform(cooc_matrix)

Постройте матрицу попарных косинусных близостей между словами, чтобы искать «соседей» по смыслу. Напомним, что косинусная близость задаётся формулой $\frac{w_i^T w_j}{||w_i||_2||w_j||_2}$

In [ ]:
# keepdims, чтобы размерность была ок
svd_embeddings_normed = svd_embeddings / (np.linalg.norm(svd_embeddings, axis=1, keepdims=True) + 1e-8)
pairwise_cosine_sim = svd_embeddings_normed @ svd_embeddings_normed.T

Выберите произвольное слово из словаря и изучите 10 его ближайших соседей по косинусной близости. Соответствует ли результат вашим ожиданиям?

Tip: для ускорения поиска можно вместо сортировки использовать [np.argpartition](https://numpy.org/doc/stable/reference/generated/numpy.argpartition.html)

In [ ]:
for word in ['cat', 'the', 'dog', 'moscow']:
    index_for_word = word_to_ind[word]
    cosine_similarities = pairwise_cosine_sim[index_for_word]
    neighbor_indices =  np.argsort(cosine_similarities)[-10:][::-1]
    neighbor_words = [ind_to_word[ind] for ind in neighbor_indices]
    print(f'{word} -> {neighbor_words}')

Теперь попробуем обучить на этих же данных модель Word2Vec. Воспользуйтесь классом [Word2Vec](https://radimrehurek.com/gensim_3.8.3/models/word2vec.html#gensim.models.word2vec.Word2Vec) из библиотеки gensim и обучите модель с размерностью векторов 300, min_count=1; остальные гиперпараметры можно оставить стандартными.

In [ ]:
from gensim.models import Word2Vec
word2vec = Word2Vec(sentences=[line.split() for line in lines], size=300, min_count=1)

Проверим качество полученной модели всем известным способом: попробуем найти наиболее близкий вектор к результату арифметической операции king-man+woman. Если вы получили не совсем то, что ожидали, то в чём могут заключаться причины этого?

In [ ]:
word2vec.wv.most_similar_cosmul(positive=['king','woman'],negative=['man'])

Помимо обучения моделей мы можем [загрузить готовые](https://radimrehurek.com/gensim_3.8.3/downloader.html) посредством той же библиотеки Gensim. Загрузим [GloVe](https://nlp.stanford.edu/projects/glove/)-векторы и посмотрим, насколько хорошо с их помощью получается искать аналогии:

In [2]:
import gensim.downloader
glove = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [3]:
glove.wv.most_similar_cosmul(positive=['tallest', 'long'],negative=['tall'])

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('longest', 0.9440028667449951),
 ('shortest', 0.7883565425872803),
 ('time', 0.7851566076278687),
 ('since', 0.77671879529953),
 ('decades', 0.7765405774116516),
 ('world', 0.7761632204055786),
 ('ever', 0.7749701142311096),
 ('decade', 0.7667874097824097),
 ('country', 0.7660934329032898),
 ('biggest', 0.7642965912818909)]

In [4]:
glove.wv.most_similar_cosmul(positive=['paris', 'russia'],negative=['france'])

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('moscow', 0.9874311089515686),
 ('russian', 0.8762803673744202),
 ('kremlin', 0.8266472816467285),
 ('helsinki', 0.8232425451278687),
 ('petersburg', 0.8097580075263977),
 ('kiev', 0.7989339828491211),
 ('putin', 0.7961435914039612),
 ('tbilisi', 0.7937439680099487),
 ('interfax', 0.7934480905532837),
 ('yeltsin', 0.790027379989624)]

In [5]:
glove.wv.most_similar_cosmul(positive=['king', 'woman'],negative=['man'])

/Users/antonina.goryacheva/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('queen', 0.9199351072311401),
 ('princess', 0.8403170108795166),
 ('throne', 0.8287888765335083),
 ('monarch', 0.8201609253883362),
 ('elizabeth', 0.8025429248809814),
 ('daughter', 0.7933654189109802),
 ('mother', 0.7825508117675781),
 ('kalākaua', 0.7787636518478394),
 ('kingdom', 0.777129590511322),
 ('wife', 0.7694059610366821)]

Визуализируем представления 200 самых частых слов в нашем датасете посредством двух методов понижения размерности — PCA и t-SNE:

In [6]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
most_common_words = [word for word, freq in vocab.most_common(200) if word in glove]
common_word_embeddings = np.stack([glove[word] for word in most_common_words if word in glove], axis=0)

In [ ]:
pca_representations = PCA(n_components=2,random_state=0).fit_transform(common_word_embeddings)
tsne_representations = TSNE(n_components=2,perplexity=25,random_state=0).fit_transform(common_word_embeddings)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(pca_representations[:,0],pca_representations[:,1])

for i, word in enumerate(most_common_words):
    plt.text(pca_representations[i,0]+0.05,pca_representations[i,1]+0.05,word)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(tsne_representations[:,0],tsne_representations[:,1])

for i, word in enumerate(most_common_words):
    plt.text(tsne_representations[i,0]+0.05,tsne_representations[i,1]+0.05,word)

Далее мы воспользуемся библиотекой [fastText](https://github.com/facebookresearch/fastText) от Facebook Research. Она интересна тем, что позволяет очень быстро обучать векторы слов посредством CLI-утилиты, а полученные векторы за счёт суммирования по n-граммам получаются устойчивы к опечаткам или другим небольшим изменениям слов. 

Установим эту библиотеку (если у вас её ещё нет), а также загрузим предобученную модель.

In [ ]:
!pip install -q fasttext

In [ ]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')

Убедимся, что модель устойчива к опечаткам: попробуйте взять англоязычное слово и посмотрите, есть ли для него вектор в модели, даже если совершить несколько орфографических ошибок.

In [ ]:
# your code here

Теперь попробуем решить задачу анализа тональности отзывов на фильмы на датасете IMDb. Код ниже скачивает данные и загружает их в pandas.DataFrame:

In [ ]:
!wget -q -nc https://github.com/LawrenceDuan/IMDb-Review-Analysis/blob/master/IMDb_Reviews.csv?raw=true -O reviews.csv

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
reviews=pd.read_csv('reviews.csv')

In [ ]:
reviews_train, reviews_test=train_test_split(reviews,test_size=1000,random_state=0)

Постройте матрицы X_train, X_test, Y_train, Y_test, содержащие усреднённые fastText-эмбеддинги слов каждого обзора и метки классов, соответствующие окраске обзора (столбец sentiment).

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_enc=LabelEncoder()

# your code here

В завершение обучим модель логистической регрессии на векторах отзывов. Как видим, даже довольно простой классификатор, не учитывающий порядок слов, может работать довольно неплохо за счёт информативных признаков.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model=LogisticRegression(max_iter=500).fit(X_train,Y_train)
accuracy_score(model.predict(X_test),Y_test)